Agriculture +

Professional Services +

Design & Engineering +

Ecommerce +

Education +

Energy +

Entertainment +

Financial Services +

Healthcare +

Industrial +

Insurance +

Legal +

Manufacturing +

Media +

Non Profit +

Retail +

Technology +

Travel and Hospitality +

Real Estate +

# Load data

## 1. Load all scrapped companies

In [1]:
import pandas as pd
import numpy as np

scrapped_companies = pd.read_csv('../data/companies-with-linkedin-data.csv', index_col=0)
scrapped_companies.count()

Associated Company ID    612
Associated Company       612
Industry                 291
Deal Name                612
Deal Stage               612
LinkedIn description     600
LinkedIn size            584
LinkedIn specialties     427
LinkedIn established     495
LinkedIn id              612
LinkedIn url             612
LinkedIn Industry        596
dtype: int64

### Load unrecognized companies but with Industries

In [2]:
unrecognized_companies = pd.read_csv('../data/unrecognized-companies.csv', index_col=0)
unrecognized_companies = unrecognized_companies[~unrecognized_companies['Industry'].isnull()]
unrecognized_companies.count()

Associated Company ID    62
Associated Company       62
Industry                 62
Deal Name                62
Deal Stage               62
dtype: int64

## 2. Concat all companies with industries

In [3]:
companies = pd.concat([scrapped_companies, unrecognized_companies], sort=False)
companies.count()

Associated Company ID    674
Associated Company       674
Industry                 353
Deal Name                674
Deal Stage               674
LinkedIn description     600
LinkedIn size            584
LinkedIn specialties     427
LinkedIn established     495
LinkedIn id              612
LinkedIn url             612
LinkedIn Industry        596
dtype: int64

# Clean data

In [4]:
df = companies.copy()

df['Industry'] = df['Industry'].str.replace(';', ' ')
df['Industry'] = df['Industry'].str.replace('Non Profit', 'NonProfit')
df['Industry'] = df['Industry'].str.replace('Food Production', 'FoodProduction')
df['Industry'] = df['Industry'].str.replace('Health Care', 'HealthCare')

# replace field that's entirely space (or empty) with NaN
df['Industry'] = df['Industry'].replace(r'^\s*$', np.nan, regex=True)

industries = df['Industry']
industries = industries.fillna('')
industries = industries.str.split()
industries = industries.apply(lambda x: ' '.join(set(x)))
industries = industries.replace('   ', '').replace('  ', '')
industries = industries.replace('', np.NaN)

df['Industry'] = industries
df['Industry'].value_counts()

E-commerce                                    66
Financial                                     43
HealthCare                                    41
Government                                    17
Insurance                                     15
Travel                                        13
NonProfit                                     10
Automotive                                     5
Sports                                         5
Technology                                     4
Gaming                                         3
Agriculture                                    2
Financial E-commerce Government HealthCare     1
Publishing                                     1
FoodProduction                                 1
E-commerce Gaming                              1
Financial Insurance                            1
NonProfit Technology                           1
NonProfit HealthCare                           1
Financial E-commerce                           1
Entertainment       

# Aggregate industry

In [5]:
def aggregate_industry_used_by_financial_department(linkedin_industry, hubspot_industry):
    industries = linkedin_industry + ' ' + hubspot_industry
    
    mappings = [
        {
            'words': ['Design', 'Software', 'Engineering', 'Maritime'],
            'industry': 'Design & Engineering'
        },
        {
            'words': ['Insurance'],
            'industry': 'Insurance'
        },
        {
            'words': ['Philanthropy', 'Non-profit', 'NonProfit', 'Government', 'Civic', 'Social', 'Military', 'Utilities'],
            'industry': 'Non Profit'
        },
        {
            'words': ['Travel', 'Hospitality'],
            'industry': 'Travel and Hospitality'
        },
        {
            'words': ['E-commerce', 'Internet', 'Import'],
            'industry': 'Ecommerce'
        },
        {
            'words': ['Oil', 'Energy', 'Renewable'],
            'industry': 'Energy'
        },
        {
            'words': ['Financial', 'Finance', 'Accounting', 'Capital', 'Investment', 'Banking'],
            'industry': 'Financial Services'
        },
        {
            'words': ['Technology', 'Telecommunications', 'Development', 'Information', 'Security', 'Biotechnology'],
            'industry': 'Technology'
        },
        {
            'words': ['Industrial', 'Construction', 'Building', 'Transportation'],
            'industry': 'Industrial'
        },
        {
            'words': ['Automotive', 'Machinery', 'Manufacturing', 'Aerospace', 'Equipment', 'Electronics'],
            'industry': 'Manufacturing'
        },
        {
            'words': ['Education', 'E-learning', 'Research', 'Training', 'Think', 'Coaching'],
            'industry': 'Education'
        },
        {
            'words': ['Entertainment', 'Museum', 'Art', 'Arts', 'Events', 'Sports', 'Sport', 'Fitness', 'Gaming', 'Game', 'Gambling'],
            'industry': 'Entertainment'
        },
        {
            'words': ['Medicine', 'Health', 'HealthCare', 'Medical'],
            'industry': 'Healthcare'
        },
        {
            'words': ['Law', 'Legal'],
            'industry': 'Legal'
        },
        {
            'words': ['Estate'],
            'industry': 'Real Estate'
        },
        {
            'words': ['Media', 'Publishing', 'Writing', 'Printing'],
            'industry': 'Media'
        },
        {
            'words': ['Human', 'Communications', 'Public', 'Management', 'Staffing', 'Logistics', 'Marketing', 'Services'],
            'industry': 'Professional Services'
        },
        {
            'words': ['Retail', 'Goods', 'Consumer', 'Furniture', 'Cosmetics', 'Fashion', 'Wine', 'Beverages', 'Restaurants'],
            'industry': 'Retail'
        },
        {
            'words': ['Agriculture', 'FoodProduction'],
            'industry': 'Agriculture'
        }
    ]

    for mapping in mappings:
        if any(word in industries for word in mapping['words']):
            return mapping['industry']

    return industries

In [6]:
df['LinkedIn Industry'] = df['LinkedIn Industry'].fillna('')
df['Industry'] = df['Industry'].fillna('')
df['Aggregated Industry'] = np.vectorize(aggregate_industry_used_by_financial_department)(df['LinkedIn Industry'], df['Industry'])

df['Aggregated Industry'] = df['Aggregated Industry'].replace(' ', np.nan)

df['Aggregated Industry'].value_counts().head(25)

Ecommerce                 99
Financial Services        71
Design & Engineering      67
Technology                66
Professional Services     49
Non Profit                46
Entertainment             39
Healthcare                36
Education                 27
Retail                    25
Insurance                 24
Media                     23
Travel and Hospitality    19
Industrial                16
Manufacturing             16
Legal                      7
Real Estate                4
Energy                     3
Agriculture                2
Name: Aggregated Industry, dtype: int64

In [7]:
df['Aggregated Industry'].value_counts().count() == 19

True

### Questions: 
 - Are Family Services, Government, Military -> Professional Services or NonProfit?
 - Restaurants -> Retail or Entertainment ?
 - Logistics, Transportation -> Professional Services or Retail or Manufactoring or Industrial?
 - Marketing & Advertisment -> Professional Services or Media ?
 - Construction, Building -> Real Estate or Industrial or Manufactoring?

In [32]:
industry_substring = 'Building'

df['LinkedIn Industry'] = df['LinkedIn Industry'].fillna('')

matching_companies = df[df['LinkedIn Industry'].str.contains(industry_substring)]
matching_companies[['Associated Company', 'LinkedIn url', 'LinkedIn Industry', 'Aggregated Industry']].head(25)

,Associated Company,LinkedIn url,LinkedIn Industry,Aggregated Industry
527,G-SMATT America,https://www.linkedin.com/company/g-smattamerica/about/,Building Materials,Industrial
580,Travis Perkins,https://www.linkedin.com/company/travis-perkins/about/,Building Materials,Industrial


# Save results

## Add hubspot url for Joe

In [9]:
pd.set_option('display.max_colwidth', 80)

df['Hubspot url']='https://app.hubspot.com/contacts/4012159/company/'+df['Associated Company ID'].str.replace(r'[0-9]+, ', '')
df['Hubspot url'].head(135).tail(10)

172    https://app.hubspot.com/contacts/4012159/company/726951995
173    https://app.hubspot.com/contacts/4012159/company/726951996
174    https://app.hubspot.com/contacts/4012159/company/726951998
175    https://app.hubspot.com/contacts/4012159/company/726952001
176    https://app.hubspot.com/contacts/4012159/company/726952002
177    https://app.hubspot.com/contacts/4012159/company/726952042
178    https://app.hubspot.com/contacts/4012159/company/726952210
179    https://app.hubspot.com/contacts/4012159/company/726952005
180    https://app.hubspot.com/contacts/4012159/company/726952006
183    https://app.hubspot.com/contacts/4012159/company/726952014
Name: Hubspot url, dtype: object

In [10]:
df_to_save = df[~df['Aggregated Industry'].isnull()].reindex()

df_to_save = df_to_save[['Associated Company ID', 'Associated Company', 'Hubspot url', 'LinkedIn url', 'LinkedIn size', 'LinkedIn specialties' , 'LinkedIn Industry' , 'Industry', 'Aggregated Industry', 'Deal Stage']]

df_to_save.to_csv('../data/companies-with-linkedin-data-and-aggregated-findep-industry.csv', index=True)

df_to_save

,Associated Company ID,Associated Company,Hubspot url,LinkedIn url,LinkedIn size,LinkedIn specialties,LinkedIn Industry,Industry,Aggregated Industry,Deal Stage
0,1005828045,Big Radical Limited,https://app.hubspot.com/contacts/4012159/company/1005828045,https://www.linkedin.com/company/big-radical/about/,11-50,"Innovation, Consultancy, design sprints, technology, mobile products, strategy",Management Consulting,NonProfit,Non Profit,0
1,1006470667,psygro.co.za,https://app.hubspot.com/contacts/4012159/company/1006470667,https://www.linkedin.com/company/psygro/about/,51-200,NaN,Mental Health Care,,Healthcare,0
2,1006849789,Care Coordination Systems,https://app.hubspot.com/contacts/4012159/company/1006849789,https://www.linkedin.com/company/beach-group/about/,2-10,"Community Health, Pathways, HUB, Pathways Connect Software, Care Transitions...",Hospital & Health Care,HealthCare,Healthcare,0
3,1011074056,KOS Services LLC,https://app.hubspot.com/contacts/4012159/company/1011074056,https://www.linkedin.com/company/kos-services-llc/about/,201-500,NaN,Medical Practice,,Healthcare,0
5,1015911460,Jellyvision,https://app.hubspot.com/contacts/4012159/company/1015911460,https://www.linkedin.com/company/jellyvision/about/,201-500,"Benefits Communication, Interactive Conversation, SaaS, Retirement Education...",Information Technology & Services,,Technology,0
...,...,...,...,...,...,...,...,...,...,...
612,727320926,GrowLogics,https://app.hubspot.com/contacts/4012159/company/727320926,NaN,NaN,NaN,,E-commerce,Ecommerce,0
637,727321131,Engine Insights,https://app.hubspot.com/contacts/4012159/company/727321131,NaN,NaN,NaN,,E-commerce,Ecommerce,0
656,727321193,Associated Software,https://app.hubspot.com/contacts/4012159/company/727321193,NaN,NaN,NaN,,Financial,Financial Services,0
716,734925936,Rwanda Ministry of Health,https://app.hubspot.com/contacts/4012159/company/734925936,NaN,NaN,NaN,,Government,Non Profit,0


## Save comparison data

In [11]:
companies = df

completed_deals = pd.read_csv('../data/completed-or-current-deails-2019-10-29.csv')
lost_deals = pd.read_csv('../data/lost-deals-2019-10-28-1.csv')

completed_deals['Deal Stage'] = '1'
lost_deals['Deal Stage'] = '0'

completed_with_found_companies = pd.merge(completed_deals, companies, on='Associated Company ID')
completed_with_found_companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407 entries, 0 to 406
Data columns (total 26 columns):
Deal ID                   407 non-null int64
Deal Name_x               407 non-null object
Deal owner                402 non-null object
Deal Stage_x              407 non-null object
Sales Lead                332 non-null object
Designated Office         379 non-null object
Director                  362 non-null object
Industry_x                168 non-null object
Source                    200 non-null object
Associated Company ID     407 non-null object
Associated Company_x      407 non-null object
Associated Contact IDs    226 non-null object
Associated Contacts       225 non-null object
Associated Company_y      407 non-null object
Industry_y                407 non-null object
Deal Name_y               407 non-null object
Deal Stage_y              407 non-null object
LinkedIn description      342 non-null object
LinkedIn size             333 non-null object
LinkedIn specialties  

In [12]:
lost_with_found_companies = pd.merge(lost_deals, companies, on='Associated Company ID')
lost_with_found_companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 833 entries, 0 to 832
Data columns (total 27 columns):
Deal ID                   833 non-null int64
Deal Name_x               833 non-null object
Deal owner                789 non-null object
Deal Stage_x              833 non-null object
Director                  528 non-null object
Sales Lead                574 non-null object
Designated Office         576 non-null object
Industry_x                193 non-null object
Source                    200 non-null object
Business Lines            718 non-null object
Associated Company ID     833 non-null object
Associated Company_x      833 non-null object
Associated Contact IDs    393 non-null object
Associated Contacts       392 non-null object
Associated Company_y      833 non-null object
Industry_y                833 non-null object
Deal Name_y               833 non-null object
Deal Stage_y              833 non-null object
LinkedIn description      712 non-null object
LinkedIn size         

In [13]:
column_name = 'Aggregated Industry'
def prepare_column_comparison(completed_deals, lost_deals, column_name):
    completed_by_column_count = completed_deals.groupby(column_name).size().sort_index().reset_index()
    completed_by_column_count.columns = [column_name, 'Closed deals']

    lost_by_column_count = lost_deals.groupby(column_name).size().sort_index().reset_index()
    lost_by_column_count.columns = [column_name, 'Lost deals']

    comparison = pd.merge(completed_by_column_count, lost_by_column_count, on=column_name)

    comparison.index = comparison[column_name].str.len()
    comparison = comparison.sort_index().reset_index(drop=True)
    return comparison

industry_comparison = prepare_column_comparison(completed_with_found_companies, lost_with_found_companies, column_name)
industry_comparison.head(20)

,Aggregated Industry,Closed deals,Lost deals
0,Legal,4,9
1,Media,9,25
2,Energy,6,1
3,Retail,3,34
4,Insurance,38,29
5,Ecommerce,62,136
6,Education,12,33
7,Healthcare,15,30
8,Industrial,5,24
9,Technology,17,75


In [14]:
number_of_companies_with_industries = companies[column_name].count()
number_of_companies_with_industries

639

In [15]:
industry_comparison.to_csv('../data/tmp-industry-findep-comparison.csv', index=True)

# Add director to companies

In [16]:
all_deals = pd.read_csv('../data/all-deals.csv', index_col=0)
all_deals.head()

,Deal ID,Deal Name,Deal owner,Deal Stage,Sales Lead,Designated Office,Director,Industry,Source,Associated Company ID,Associated Company,Associated Contact IDs,Associated Contacts,Business Lines
0,1126055965,American Family Insurance - Shark,Kevin Kotowski,1,Dennis Moore,Madison,Kevin Kotowski,Insurance,None,846172243,American Family Insurance,NaN,NaN,NaN
1,362032710,SSE,Jim Suchy,1,Margaret Pagel,London,Nick Dyer,Government,Existing client referral,727202247,Arthur D Little,NaN,NaN,NaN
2,941965426,Confluency - X Factor,Brad Ediger,1,Dennis Moore,Chicago,Dennis Moore,Government,Cold Call,2338059811,Confluency,30437651,Mason Throneburg,NaN
3,905986805,Reverb - Holistic On-boarding/Training/Mentor Coaching,Ryan Verner,1,Jenn Imamura,Chicago,Ryan Verner,E-commerce,Website,629633500,Reverb.com LLC,"37461401, 35715851, 38635601","Molly Rudberg, Kyle Crum, Jason Wain",NaN
4,876852247,Maintenance and Consulting Services,Kim Carten,1,Jenn Imamura,Los Angeles,Malcolm Newsome,NaN,Upsell,727079949,CMD-IT,NaN,NaN,NaN


In [17]:
all_deals['Director'] = all_deals['Director'].fillna('')

def agg_unique_directors(row_of_joining_columns):
    unique_values = set(row_of_joining_columns)
    if unique_values.__len__() > 1 and unique_values.__contains__(''):
        unique_values.remove('')
    return ', '.join(unique_values)

grouped_by_company_id = all_deals.groupby(['Associated Company ID'], as_index=False)\
    .agg({
        'Associated Company': 'first',
        'Director': lambda x: agg_unique_directors(x)
        })

grouped_by_company_id['Director'].value_counts().head(45)
#grouped_by_company_id.head(25)

                                                                     242
Jim Suchy                                                            130
Doug Bradbury                                                         93
Dave Moore                                                            58
Kevin Kotowski                                                        51
Malcolm Newsome                                                       41
Gustin Prudner (Deactivated User)                                     28
Connor Mendenhall                                                     25
Jim Remsik (Deactivated User)                                         12
Ryan Verner                                                           10
Ray Hightower (Deactivated User)                                       9
Brad Ediger                                                            9
Malcolm Newsome, Doug Bradbury                                         8
heather corallo                                    

In [18]:
deals_directors = grouped_by_company_id[['Associated Company ID', 'Director']]

deals_directors.head()

,Associated Company ID,Director
0,1005828045,Nick Dyer
1,1006470667,
2,1006849789,Doug Bradbury
3,1011074056,
4,1013701003,


In [19]:
df = pd.merge(deals_directors, df, on='Associated Company ID')
df.head()

,Associated Company ID,Director,Associated Company,Industry,Deal Name,Deal Stage,LinkedIn description,LinkedIn size,LinkedIn specialties,LinkedIn established,LinkedIn id,LinkedIn url,LinkedIn Industry,Aggregated Industry,Hubspot url
0,1005828045,Nick Dyer,Big Radical Limited,NonProfit,Web Community Project,0,Big Radical is a breakthrough design and innovation agency and part of the E...,11-50,"Innovation, Consultancy, design sprints, technology, mobile products, strategy",2017.0,big-radical,https://www.linkedin.com/company/big-radical/about/,Management Consulting,Non Profit,https://app.hubspot.com/contacts/4012159/company/1005828045
1,1006470667,,psygro.co.za,,Trudy Tanner - 8thlight.com contact,0,Psygro (Psychiatrische Groepspraktijk) biedt basis- en specialistische Geest...,51-200,NaN,NaN,psygro,https://www.linkedin.com/company/psygro/about/,Mental Health Care,Healthcare,https://app.hubspot.com/contacts/4012159/company/1006470667
2,1006849789,Doug Bradbury,Care Coordination Systems,HealthCare,CCS Pathways -- Admin Modules and Web Education Registration,0,Pathways Community HUB model Leading the Way to Building Better Community He...,2-10,"Community Health, Pathways, HUB, Pathways Connect Software, Care Transitions...",2013.0,beach-group,https://www.linkedin.com/company/beach-group/about/,Hospital & Health Care,Healthcare,https://app.hubspot.com/contacts/4012159/company/1006849789
3,1011074056,,KOS Services LLC,,Erin Steinhardt - 8thlight.com contact,0,"KOS SERVICES, LLC is a medical practice company based out of 350 N CLARK ST,...",201-500,NaN,NaN,kos-services-llc,https://www.linkedin.com/company/kos-services-llc/about/,Medical Practice,Healthcare,https://app.hubspot.com/contacts/4012159/company/1011074056
4,1015911460,,Jellyvision,,Mike Else - 8thlight.com contact,0,Jellyvision is an award-winning technology company whose interactive softwar...,201-500,"Benefits Communication, Interactive Conversation, SaaS, Retirement Education...",2001.0,jellyvision,https://www.linkedin.com/company/jellyvision/about/,Information Technology & Services,Technology,https://app.hubspot.com/contacts/4012159/company/1015911460


In [20]:
company_directors = df[['Associated Company ID', 'Associated Company', 'Director']].sort_values('Associated Company')
company_directors.head()

,Associated Company ID,Associated Company,Director
391,727202258,@esparkslearning,
52,2216463622,42 North Dental LLC,Brad Ediger
575,727321320,528 Records,Kevin Kotowski
102,701404041,8to18,"Malcolm Newsome, Doug Bradbury"
673,995352975,ACCC Insurance,Jim Remsik (Deactivated User)


### Saving only directors column

In [21]:
df_to_save = company_directors.reset_index()
df_to_save = df_to_save[['Associated Company ID', 'Associated Company', 'Director']]
df_to_save.to_csv('../data/companies-directors.csv', index=True)

df_to_save

,Associated Company ID,Associated Company,Director
0,727202258,@esparkslearning,
1,2216463622,42 North Dental LLC,Brad Ediger
2,727321320,528 Records,Kevin Kotowski
3,701404041,8to18,"Malcolm Newsome, Doug Bradbury"
4,995352975,ACCC Insurance,Jim Remsik (Deactivated User)
...,...,...,...
669,848746315,"thoughtbot, inc.",Jim Suchy
670,865451460,voxgig,Jim Suchy
671,726952195,vrsus,Malcolm Newsome
672,775616077,youryolk.com,


### Saving companies with aggregated industries + directors

In [22]:
df_to_save = df.reindex()
df_to_save = df_to_save[['Associated Company ID', 'Associated Company', 'Director', 'Hubspot url', 'LinkedIn url', 'LinkedIn size', 'LinkedIn specialties' , 'LinkedIn Industry' , 'Industry', 'Aggregated Industry', 'Deal Stage']]

df_to_save.to_csv('../data/companies-with-linkedin-data-and-aggregated-findep-industry-and-directors.csv', index=True)

df_to_save

,Associated Company ID,Associated Company,Director,Hubspot url,LinkedIn url,LinkedIn size,LinkedIn specialties,LinkedIn Industry,Industry,Aggregated Industry,Deal Stage
0,1005828045,Big Radical Limited,Nick Dyer,https://app.hubspot.com/contacts/4012159/company/1005828045,https://www.linkedin.com/company/big-radical/about/,11-50,"Innovation, Consultancy, design sprints, technology, mobile products, strategy",Management Consulting,NonProfit,Non Profit,0
1,1006470667,psygro.co.za,,https://app.hubspot.com/contacts/4012159/company/1006470667,https://www.linkedin.com/company/psygro/about/,51-200,NaN,Mental Health Care,,Healthcare,0
2,1006849789,Care Coordination Systems,Doug Bradbury,https://app.hubspot.com/contacts/4012159/company/1006849789,https://www.linkedin.com/company/beach-group/about/,2-10,"Community Health, Pathways, HUB, Pathways Connect Software, Care Transitions...",Hospital & Health Care,HealthCare,Healthcare,0
3,1011074056,KOS Services LLC,,https://app.hubspot.com/contacts/4012159/company/1011074056,https://www.linkedin.com/company/kos-services-llc/about/,201-500,NaN,Medical Practice,,Healthcare,0
4,1015911460,Jellyvision,,https://app.hubspot.com/contacts/4012159/company/1015911460,https://www.linkedin.com/company/jellyvision/about/,201-500,"Benefits Communication, Interactive Conversation, SaaS, Retirement Education...",Information Technology & Services,,Technology,0
...,...,...,...,...,...,...,...,...,...,...,...
669,966759785,"Fast Radius, Inc.",Jim Remsik (Deactivated User),https://app.hubspot.com/contacts/4012159/company/966759785,https://www.linkedin.com/company/fast-radius/about/,51-200,NaN,Mechanical Or Industrial Engineering,E-commerce,Design & Engineering,0
670,988061987,asktili.com,Jim Suchy,https://app.hubspot.com/contacts/4012159/company/988061987,https://www.linkedin.com/company/asktili/about/,201-500,"homemovers, utilities, propertymanagement, energy, broadband, Sky TV, lettin...",Consumer Services,,Professional Services,0
671,988677932,"Carco Technologies, Inc.",Gustin Prudner (Deactivated User),https://app.hubspot.com/contacts/4012159/company/988677932,https://www.linkedin.com/company/carco-technologies-inc./about/,11-50,NaN,Information Technology & Services,,Technology,0
672,992979974,"Weathergram, Inc dba Switchboard",Malcolm Newsome,https://app.hubspot.com/contacts/4012159/company/992979974,https://www.linkedin.com/company/weathergram-inc./about/,11-50,NaN,Information Technology & Services,,Technology,1;1
